In [1]:
import pandas as pd
import seaborn as sns
import string
import numpy as np
import spacy
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,RobustScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
old_data=pd.read_csv("tweet.csv")
new_data=pd.read_csv("twt_classs_nlp.csv")


In [4]:
old_data.head()

,tweets,class
0,Be aware dirty step to get money #staylight ...,figurative
1,#sarcasm for #people who don't understand #diy...,figurative
2,@IminworkJeremy @medsingle #DailyMail readers ...,figurative
3,@wilw Why do I get the feeling you like games?...,figurative
4,-@TeacherArthurG @rweingarten You probably jus...,figurative


In [5]:
old_data=old_data.drop(old_data[old_data.tweets.duplicated()].index)


In [6]:
old_data.tail(10)

,tweets,class
81398,Whoa. Didn't see that coming. #sarcasm,sarcasm
81399,An hour busride home soakin wet is the best th...,sarcasm
81400,Nothing like having someone to talk to about y...,sarcasm
81401,Looks like aj brown is having a bad game... He...,sarcasm
81402,oh I love 4 hour bus rides #sarcasm,sarcasm
81403,Photo: Image via We Heart It http://t.co/ky8Nf...,sarcasm
81404,I never knew..I better put this out to the Uni...,sarcasm
81405,hey just wanted to say thanks @ puberty for le...,sarcasm
81406,I'm sure coverage like the Fox News Special “T...,sarcasm
81407,@skeyno16 at u13?! I won't believe it until I ...,sarcasm


In [7]:
old_data.head()

,tweets,class
0,Be aware dirty step to get money #staylight ...,figurative
1,#sarcasm for #people who don't understand #diy...,figurative
2,@IminworkJeremy @medsingle #DailyMail readers ...,figurative
3,@wilw Why do I get the feeling you like games?...,figurative
4,-@TeacherArthurG @rweingarten You probably jus...,figurative


In [8]:
def separte_hash(txt):
  temp=txt.split(" ")
  remove_word=['sarcasm','irony','ironic','sarcastic']
  temp=[word for word in temp if not word in remove_word]
  temp=' '.join(temp)
  return temp

In [9]:
new_data.isnull().sum()

Unnamed: 0          0
clean_tweet         4
class               0
sentiment           0
class_lle_encode    0
y_hat_pred          0
dtype: int64

In [10]:
new_data=new_data.dropna()
new_data=new_data.reset_index()
new_data.head()

,index,Unnamed: 0,clean_tweet,class,sentiment,class_lle_encode,y_hat_pred
0,0,0,aware dirty step get money staylight staywhite...,figurative,negative,0,3
1,1,1,sarcasm people do not understand diy artattack,figurative,nutral,0,3
2,2,2,dailymail reader sensible always shock sarcast...,figurative,negative,0,3
3,3,3,get feel like game sarcasm,figurative,nutral,0,3
4,4,4,probably miss text sarcastic,figurative,nutral,0,3


In [11]:
new_data['after_removal']=""

In [12]:
for i in range(0,len(new_data['clean_tweet'])):
  if(new_data['class'][i]=='figurative'):
    new_data['after_removal'][i]=separte_hash(new_data['clean_tweet'][i])
  else:
    new_data['after_removal'][i]=new_data['clean_tweet'][i]
new_data['after_removal']


<ipython-input-12-d1c21013fa97>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['after_removal'][i]=separte_hash(new_data['clean_tweet'][i])
<ipython-input-12-d1c21013fa97>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['after_removal'][i]=new_data['clean_tweet'][i]


0        aware dirty step get money staylight staywhite...
1                   people do not understand diy artattack
2        dailymail reader sensible always shock dailyfa...
3                                       get feel like game
4                                       probably miss text
                               ...                        
67988    photo image via heart childhood cool funny sar...
67989    never knewi well put universe lolmaybe there s...
67990    hey want say thank puberty let apart itty bitt...
67991    I m sure coverage like fox news special " the ...
67992                   u13 will not believe see p sarcasm
Name: after_removal, Length: 67993, dtype: object

In [13]:
#for i in range(0,len(new_data['clean_tweet'])):
#  if(new_data['class'][i]=='figurative'):
#    text=new_data['after_removal'][i].split(" ")
#    new_data['after_removal'][i]=separte_hash(new_data['clean_tweet'][i])
#  else:
#    new_data['after_removal'][i]=new_data['clean_tweet'][i]
#new_data['after_removal']

In [14]:
new_data.isnull().sum()

index               0
Unnamed: 0          0
clean_tweet         0
class               0
sentiment           0
class_lle_encode    0
y_hat_pred          0
after_removal       0
dtype: int64

In [15]:
data_exp=pd.DataFrame()
data_exp['old_tweet']=new_data['clean_tweet']
data_exp['new_tweet']=new_data['after_removal']
data_exp['twt_class']=new_data['class']
data_exp.head()

,old_tweet,new_tweet,twt_class
0,aware dirty step get money staylight staywhite...,aware dirty step get money staylight staywhite...,figurative
1,sarcasm people do not understand diy artattack,people do not understand diy artattack,figurative
2,dailymail reader sensible always shock sarcast...,dailymail reader sensible always shock dailyfa...,figurative
3,get feel like game sarcasm,get feel like game,figurative
4,probably miss text sarcastic,probably miss text,figurative


In [16]:
nlp=spacy.load('en_core_web_sm')
my_stop_word=stopwords.words('english')
for i in range(0,len(data_exp['new_tweet'])):
  text_tokens=(data_exp['new_tweet'][i]).split(" ")
  doc=nlp(' '.join(text_tokens))
  txt=[token.lemma_ for token in doc]
  data_exp['new_tweet'][i]=' '.join(txt)

#data1.head()

In [17]:
my_stop_word=nltk.corpus.stopwords.words('english')
my_stop_word.append('m')
my_stop_word.append('s')
for i in range(0,len(data_exp['new_tweet'])):
  text_tokens=(data_exp['new_tweet'][i]).split(" ")
  txt=[word for word in text_tokens if not word in my_stop_word]
  txt=' '.join(txt)
  data_exp['new_tweet'][i]=txt
data_exp.head()

,old_tweet,new_tweet,twt_class
0,aware dirty step get money staylight staywhite...,aware dirty step get money staylight staywhite...,figurative
1,sarcasm people do not understand diy artattack,people understand diy artattack,figurative
2,dailymail reader sensible always shock sarcast...,dailymail reader sensible always shock dailyfa...,figurative
3,get feel like game sarcasm,get feel like game,figurative
4,probably miss text sarcastic,probably miss text,figurative


In [18]:
from sklearn.feature_extraction.text import CountVectorizer


In [19]:
tweet_all=data_exp['new_tweet']
tweet_all=data_exp[data_exp['twt_class']=='figurative']['new_tweet']
cnt_vect_fig=CountVectorizer(analyzer='word',ngram_range=(1,3),max_features = 650)
X=cnt_vect_fig.fit_transform(tweet_all)
sum_words=X.sum(axis=0)

words_freq=[(word,sum_words[0,idx]) for word,idx in cnt_vect_fig.vocabulary_.items()]
words_freq=sorted(words_freq,key=lambda x:x[1] ,reverse=True)

wd_df=pd.DataFrame(words_freq)
wd_df.columns=['word','count']
#wd_df.to_csv("figurative_ngram_cntvect.csv")
wd_df.head(10)

,word,count
0,get,1591
1,go,1048
2,like,1008
3,people,986
4,love,938
5,good,883
6,make,865
7,say,857
8,day,835
9,one,804


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
#figurative
tweet_all=data_exp['new_tweet']
tfidf_fig=TfidfVectorizer(norm="l2",analyzer='word', ngram_range=(1,3), max_features = 1000)
X=tfidf_fig.fit_transform(tweet_all)
sum_words=X.sum(axis=0)

words_freq=[(word,sum_words[0,idx]) for word,idx in tfidf_fig.vocabulary_.items()]
words_freq=sorted(words_freq,key=lambda x:x[1] ,reverse=True)

wd_df=pd.DataFrame(words_freq)
wd_df.columns=['word','weight']
#wd_df.to_csv("figurative_ngram_tfidf.csv")
wd_df.head(10)

,word,weight
0,sarcasm,3575.984061
1,irony,2665.051318
2,get,1282.717952
3,ironic,1139.448979
4,late,1135.229286
5,love,1080.821817
6,peace,1076.547144
7,news,1073.545259
8,drug,1007.087400
9,humor,983.789810


In [21]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
data_exp['lbl_class']=data_exp['twt_class'].map({'figurative':0,'sarcasm':1,'irony':2,'regular':3})
data_exp

,old_tweet,new_tweet,twt_class,lbl_class
0,aware dirty step get money staylight staywhite...,aware dirty step get money staylight staywhite...,figurative,0
1,sarcasm people do not understand diy artattack,people understand diy artattack,figurative,0
2,dailymail reader sensible always shock sarcast...,dailymail reader sensible always shock dailyfa...,figurative,0
3,get feel like game sarcasm,get feel like game,figurative,0
4,probably miss text sarcastic,probably miss text,figurative,0
...,...,...,...,...
67988,photo image via heart childhood cool funny sar...,photo image via heart childhood cool funny sar...,sarcasm,1
67989,never knewi well put universe lolmaybe there s...,never knewi well put universe lolmaybe date up...,sarcasm,1
67990,hey want say thank puberty let apart itty bitt...,hey want say thank puberty let apart itty bitt...,sarcasm,1
67991,"I m sure coverage like fox news special "" the ...","I sure coverage like fox news special "" hidden...",sarcasm,1


In [23]:
y=data_exp['lbl_class']

In [24]:
x_train,x_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.2,random_state=43)

In [25]:
model_xg = XGBClassifier(n_estimators=500,max_depth=3,gamma=2, max_leaves=10, learning_rate=0.01)
model_xg.fit(x_train, y_train)
y_pred_train=model_xg.predict(x_train)
y_pred_test=model_xg.predict(x_test)
print("train_score",metrics.accuracy_score(y_pred_train,y_train))
print("test_score",metrics.accuracy_score(y_pred_test,y_test))

train_score 0.9877192337390154
test_score 0.9883079638208692


In [26]:
print(metrics.classification_report(y_pred_train,y_train))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     16619
           1       1.00      1.00      1.00     12325
           2       1.00      1.00      1.00     10232
           3       0.99      0.97      0.98     15218

    accuracy                           0.99     54394
   macro avg       0.99      0.99      0.99     54394
weighted avg       0.99      0.99      0.99     54394

